In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# UAM 소음 출력 레벨 (dB), 예: 100 dB
L_w = 150

# UAM의 x 위치는 500m에서 고정, y는 0m에서 1000m까지 모든 위치
x_UAM = 500
y_UAM_range = np.linspace(0, 1000, 1000)

# x와 y 좌표 범위 (1km × 1km, 1m 간격)
x_range = np.linspace(0, 1000, 1000)
y_range = np.linspace(0, 1000, 1000)

# 그리드 만들기 (x, y 좌표 매칭)
X, Y = np.meshgrid(x_range, y_range)

# 각 좌표에서 UAM과의 최소 거리 계산
def calculate_min_distance(x, y, x_UAM, y_UAM_range):
    distances = np.sqrt((x - x_UAM)**2 + (y[:, None] - y_UAM_range[None, :])**2)
    min_distance = np.min(distances, axis=1)
    return min_distance

# 소음 계산 함수
def calculate_noise(L_w, d):
    d = np.maximum(d, 1e-10)  # 거리 d가 0이 되는 것을 방지
    L_p = L_w - 20 * np.log10(d) - 8
    return L_p

# 구간 정의
x_segments = [0, 250, 500, 750, 1000]

# 각 구간에서의 평균 소음 계산 함수
def segment_average_noise(x_segments, L_w, y_range, x_UAM, y_UAM_range):
    segment_averages = []
    for i in range(len(x_segments) - 1):
        start_x, end_x = x_segments[i], x_segments[i+1]

        # 해당 구간의 모든 x 좌표에 대해 소음 계산
        x_segment = np.linspace(start_x, end_x, 100)  # 구간 내 100개 x 좌표 샘플링
        all_noises = []
        for x in x_segment:
            if start_x == 250 and end_x == 500:  # 250m ~ 500m 구간
                noise_250m = calculate_noise(L_w, calculate_min_distance(250, y_range, x_UAM, y_UAM_range))
                noise_500m = np.full_like(noise_250m, L_w)  # 500m에서의 소음은 L_w 고정
                noise_values = (noise_250m + noise_500m) / 2  # 평균 소음 계산
            elif start_x == 500 and end_x == 750:  # 500m ~ 750m 구간
                noise_500m = np.full_like(y_range, L_w)  # 500m에서의 소음은 L_w 고정
                noise_750m = calculate_noise(L_w, calculate_min_distance(750, y_range, x_UAM, y_UAM_range))
                noise_values = (noise_500m + noise_750m) / 2  # 평균 소음 계산
            else:  # 나머지 구간
                min_distances = calculate_min_distance(x, y_range, x_UAM, y_UAM_range)  # 각 y에 대해 최소 거리 계산
                noise_values = calculate_noise(L_w, min_distances)
            all_noises.append(np.mean(noise_values))  # y 축 전체 평균 소음 값

        # 구간의 평균 소음 값
        segment_avg = np.mean(all_noises)
        segment_averages.append(segment_avg)

    return segment_averages

# 구간별 평균 소음 계산
segment_averages = segment_average_noise(x_segments, L_w, y_range, x_UAM, y_UAM_range)

# CSV 파일로 출력
df = pd.DataFrame({
    'Segment Start (m)': x_segments[:-1],
    'Segment End (m)': x_segments[1:],
    'Average Noise Level (dB)': segment_averages
})
df.to_csv('segment_averages.csv', index=False)

print("CSV 파일로 결과를 저장했습니다.")
